In [1]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl huggingface_hub gradio openai

In [2]:
%%capture
%pip install --upgrade langchain

In [3]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    logging,
    BitsAndBytesConfig
)
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import requests
import datasets
import pandas as pd
import gradio as gr

In [4]:
#setting up the pipeline by loading base and adapter model
adapter_model = "naqib3110/llama-2-7b-chat-whazzat"
base_model = "meta-llama/Llama-2-7b-chat-hf"
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(base_model,
    quantization_config=quant_config,
    device_map={"": 0},
    token='')
model.config.use_cache = False
model.config.pretraining_tp = 1
model.load_adapter(adapter_model)

pipeline = pipeline(
    "text-generation", #task
    model="naqib3110/llama-2-7b-chat-whazzat",
    tokenizer="naqib3110/llama-2-7b-chat-whazzat",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=250,
    do_sample=True,
    top_k=10,
    token=''
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

adapter_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

In [5]:
#creating llm chain
template = """
           You are an expert product recommender,
           You can generate product recommendations based on simple description of users' need.
           Recommend products that are as relevant as possible .
           Recommend only 5 products.
           Do not generate additional prompt as a user
           Only generate 1 array for the asked usecase
           Return nothing but an array of 5 items. Leave out any extra words that have nothing to do with the product names.

           example,
           USER: something to clean my house
           PRODUCT RECOMMENDER: ['Vacuum Cleaner','Mop and Bucket','Broom and Dustpan','Steam Cleaner','Robot Vacuum'].

           USER: {query}
           PRODUCT RECOMMENDER:
           """
llm = HuggingFacePipeline(pipeline=pipeline,model_kwargs={'temperature':0.6})
prompt = PromptTemplate(template=template, input_variables=["query"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [6]:
#function to get model output
def gen(query):
    result = llm_chain.run(query)
    cleaned_str = result.split('.')[0].split('\n')[0]
    # print(cleaned_str)

    return cleaned_str

In [10]:
# Evaluation code using GPT-3.5-turbo from OpenAI
import csv
import pandas as pd
from openai import OpenAI

API_KEY=''
def evaluate_output(query, query_result):
  client = OpenAI(api_key=API_key)
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {
        "role": "system",
        "content":   "Act as a classifier. You will take a statement and products as input. Output 'positive' if all products are relevant to the description.  Otherwise return the products in relevant and irrelevant buckets.\n\nSample input 1:\nInput statement: something for a home workout session\nInput resut:  ['Resistance band', 'Yoga mat', 'Dumbbell set']\nOutput: Positive\n\nSample input 2:\nInput statement: metal dealy to dig food from plate\nInput result: ['stainless steel fork', 'spoon', 'Garden Fork', 'Soil Sifter', 'Garden Rake']\n\nOutput: \nrelevant: 'stainless steel fork', 'spoon'\n irrelevant: 'Garden Fork', 'Soil Sifter', 'Garden Rake'"
      },
      {
        "role": "user",
        "content": f"Input statement: {query}\nInput resut:  {query_result}"
      }
    ],
    temperature=0.6,
    max_tokens=250,
    top_p=0.6,
    frequency_penalty=0,
    presence_penalty=0
  )

  return response

In [15]:
#extracting test samples into input_arr
!unzip /content/model_dataset.zip
dataset = datasets.load_from_disk("/content/content/dataset.hf")
test_arr=[]
input_arr=[]
for i in range(61):
  test_arr.append(dataset['test'][i]['text'])
for str in test_arr:
  input_arr.append(str.replace('<s>[INST]','').split('[/INST]')[0])

Archive:  /content/model_dataset.zip
replace content/dataset.hf/test/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [17]:
#getting model output on all test samples
outputs_arr=[]
for v,i in enumerate(input_arr):
  print(v)
  response = gen(i)
  print(response)
  outputs_arr.append(response)

0
 ['KitchenAid Stand Mixer', 'All-Clad Stainless Steel 10-Piece Cookware Set', 'Instant Pot Pressure Cooker', 'Keurig K-Mini Plus Coffee Maker', 'Anolon Advanced Hard-An
1
 ['Faux Fur Blanket', 'Plush Throw Blanket', 'Wool Blanket', 'Slippers', 'Candle']
2
 ['Grill Pan', 'Fish Grill Mat', 'Fish Skewers', 'Fish Tray', 'BBQ Gloves']
3
 ['Candy Making Kit', 'Chocolate Melter', 'Candy Thermometer', 'Flavor Extracts', 'Candy Mold']
4
 ['Rice Noodles', 'Tamarind Paste', 'Peanut Butter', 'Coconut Milk', 'Lime Juice']
5
 ['Pan', 'Spatula', 'Measuring Cups', 'Baking Powder', 'Syrup Bottle']
6
 ['Desk Organizer', 'Task Light', 'Whiteboard', 'Paperweight', 'Inbox Organizer']
7
 ['Green Tea Bags', 'Aloe Vera Gel', 'Lemon Squeezer', 'Aromatherapy Essential Oil', 'Morning Sunrise Simulation Lamp']
8
 ['Patio Furniture Set', 'Outdoor Speakers', 'Garden Umbrella', 'Outdoor Heaters', 'Barbecue Grill']
9


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 ['Portable Blanket and Pillow', 'Folding Table and Chairs', 'Waterproof Blanket', 'Cooler Bag', 'Outdoor Throw Pillows']
10


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 ['Comfortable Chair', 'Laptop Stand', 'Noise-Cancelling Headphones', 'Whiteboard and Markers', 'Focused Study Light']
11


...

 ['Avocado'], ['Lime'], ['Jalapeno Pepper'], ['Cilantro'], ['Cotija Cheese']
59


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 ['Gardening Gloves', 'Hand Trowel', 'Rake', 'Watering Can', 'Garden Pruner']
60


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 ['Power Drill', 'Circular Saw', 'Reciprocating Saw', 'Tile Cutter', 'Level']


In [28]:
#evaluating all inputs and respective outputs using gpt-3.5-turbo
gpt_checks=[]
df = pd.DataFrame(columns=['input', 'output', 'class'])
for input,output in zip(input_arr,outputs_arr):
  check = evaluate_output(input,output)
  gpt_checks.append(check)
  content = check.choices[0].message.content.lower()
  print(f"'input': {input}, 'output': {output}, 'class': {content}")
  df = df.append({'input': input, 'output': output, 'class': content}, ignore_index=True)

'input': something for a new cooking hobby, 'output':  ['KitchenAid Stand Mixer', 'All-Clad Stainless Steel 10-Piece Cookware Set', 'Instant Pot Pressure Cooker', 'Keurig K-Mini Plus Coffee Maker', 'Anolon Advanced Hard-An, 'class': output: positive


<ipython-input-28-a1bca3dd1760>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'input': input, 'output': output, 'class': content}, ignore_index=True)


'input': need something cozy for a winter evening at home, 'output':  ['Faux Fur Blanket', 'Plush Throw Blanket', 'Wool Blanket', 'Slippers', 'Candle'], 'class': output: positive


<ipython-input-28-a1bca3dd1760>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'input': input, 'output': output, 'class': content}, ignore_index=True)


'input': something for grilling fish, 'output':  ['Grill Pan', 'Fish Grill Mat', 'Fish Skewers', 'Fish Tray', 'BBQ Gloves'], 'class': output: positive


...

<ipython-input-28-a1bca3dd1760>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'input': input, 'output': output, 'class': content}, ignore_index=True)


'input': something for gardening, 'output':  ['Gardening Gloves', 'Hand Trowel', 'Rake', 'Watering Can', 'Garden Pruner'], 'class': output: positive


<ipython-input-28-a1bca3dd1760>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'input': input, 'output': output, 'class': content}, ignore_index=True)


'input': something for a diy home improvement project, 'output':  ['Power Drill', 'Circular Saw', 'Reciprocating Saw', 'Tile Cutter', 'Level'], 'class': output: positive


<ipython-input-28-a1bca3dd1760>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'input': input, 'output': output, 'class': content}, ignore_index=True)


In [29]:
df.to_csv('/content/evaluation.csv')